In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import requests
import json
import bs4
from bs4 import BeautifulSoup

Now, we get the merge data and do some calculation based on it. We then get the desired features for us to do the analysis.

In [2]:
merge_tab = pd.read_csv('merge2015.csv')
merge_tab.shape

(422, 71)

First, we transform the date of brith of actors and directors into date format, then substract the released date from their birthday to get the age of directors and actors when the movie was released (they are called 'Aage' and 'Dage'). We think this is an important feature because age, to some extent, implies the experience of the director/actor.

In [3]:
import dateutil.parser as parser
from datetime import datetime

for i in range(len(merge_tab.Released)):
    if merge_tab.Released[i] == 'False':
        merge_tab.Released[i] = '1929-08-20 00:00:00'

RD = []
DDOB = []
ADOB = []
for i in range(len(merge_tab.Released)):
    try:
        RD.append(parser.parse(merge_tab.Released[i]))
        DDOB.append(parser.parse(merge_tab.D_DOB[i]))
        ADOB.append(parser.parse(merge_tab.A_DOB[i]))
    except:
        pass
merge_tab['RD']=RD
merge_tab['DDOB']=DDOB
merge_tab['ADOB']=ADOB
merge_tab['Dage'] = (merge_tab.RD - merge_tab.DDOB).values/np.timedelta64(1, 'D')/365.25
merge_tab['Aage'] = (merge_tab.RD - merge_tab.ADOB).values/np.timedelta64(1, 'D')/365.25



/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We drop the unnecessary columns from the merged data frame and remove the comma in 'imdbVotes'. The purpose of this is to transform 'imdbVotes' into numeric .

In [5]:
merge_tab = merge_tab.drop(['Unnamed: 0','Awards','Metascore','D_wins','A_wins','RD','A_age','D_age','DDOB','ADOB'], axis=1)

#merge_tab['imdbVotes'] = merge_tab.imdbVotes.apply(lambda x: x.replace(',',''))

Now we have the ages for each of the director/actor in each movie, we need to calculate an weighted mean of the ages. We choose the weights to be the 'credits' of the director/actor. 'Credits' here are how many movies the actor has been the head actor for, or how many movies the director has directed. Besides age, we are also interested in the ratings of each movie. Currently, we have the user rating for each movie from both IMDB and Rotten Tomatos. We, again, calculate the weighted average of these two ratings by the reviews counts from each website.


In [6]:
def A_w_mean(df):
    return np.average(df.Aage, weights=df.A_credits)
def D_w_mean(df):
    return np.average(df.Dage, weights=df.D_credits)
def mean_rating(tab):
    w1 = [float(i) for i in tab.imdbVotes]
    w2 = [float(i) for i in tab.tomatoUserReviews]
    if any(np.isnan(tab.tomatoUserRating)):
        mean_rate = np.average(tab.imdbRating,  weights=w1)
    else:
        mean_rate = (np.average(tab.imdbRating,  weights=w1) + 2*np.average(tab.tomatoUserRating,  weights=w2))/2
    return mean_rate

The function 'clean_table' calculates the weighted average of ages, credits and rating.

In [7]:
def clean_table(tab):
    movies = merge_tab.groupby('imdbID')
    A_mean_age = movies.apply(A_w_mean)
    D_mean_age = movies.apply(D_w_mean)
    A_mean_credit = movies.apply(lambda x: np.average(x.A_credits))
    D_mean_credit = movies.apply(lambda x: np.average(x.D_credits))
    rating_mean = movies.apply(mean_rating)
    dic = {'A_mean_age': A_mean_age, 'D_mean_age': D_mean_age, 'A_mean_credit':A_mean_credit, 'D_mean_credit':D_mean_credit,
          'mean_rate': rating_mean}
    output = pd.DataFrame(dic)
    return output 

In [8]:
for i, r in merge_tab.iterrows():
    if type(r.imdbVotes) == str:
        merge_tab['imdbVotes'][i]= r.imdbVotes.replace(',','')

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
for i, r in merge_tab.iterrows():
    if type(r.tomatoUserReviews) == str:
        merge_tab['tomatoUserReviews'][i]= r.tomatoUserReviews.replace(',','')

The following are the result of calculation, each row corresponds to a movie.

In [10]:
merge_tab2 = clean_table(merge_tab).reset_index()
merge_tab2.head()

,imdbID,A_mean_age,A_mean_credit,D_mean_age,D_mean_credit,mean_rate
0,tt0369610,47.432927,73.000000,38.743326,7,7.50
1,tt0470752,31.544133,31.000000,45.308693,2,7.85
2,tt0478970,51.516363,67.333333,51.036277,26,7.90
3,tt0810819,30.397230,25.500000,43.230664,18,6.75
4,tt0884732,32.953663,56.333333,39.129363,5,6.95


Since in the original table, each movie has three row, and each row corresponds to a different actor/director combination, we need to combine these rows into one single row for each movies. Therefore we remove the duplicate rows in the table. After we do that, we merge the original table and the calcualted table.

In [11]:
#merge_tab

In [12]:
merge_tab3 = pd.merge(merge_tab, merge_tab2, left_on = 'imdbID', right_on='imdbID', how='left')
merge_tab3 = merge_tab3.drop(['D_birthplace','D_DOB','D_name','D_credits','A_birthplace','A_DOB','A_name','A_credits',
              'Actors', 'Director', 'Dage', 'Aage'], axis=1)
merge_tab3 = merge_tab3.drop_duplicates()

In [15]:
merge_tab3.to_csv('final_table2015.csv', encoding='utf-8')

In [16]:
merge_tab3.head()

,imdbID,Title,Released,Country,Language,Rated,Runtime,Year,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,Nominated Oscars,Won Oscars,Other wins,Other nominations,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,N/A,Romance,Sci-Fi,Sport,Thriller,War,Western,Director Nominated Oscars,Director Won Oscars,Actor Nominated Oscars,Actor Won Oscars,wiki_url_new,imdb_link_new,rt_link_new,budget_USD,box_USD,A_mean_age,A_mean_credit,D_mean_age,D_mean_credit,mean_rate
0,tt0369610,Jurassic World,2015-06-12 00:00:00,USA,English,PG-13,124,2015,7.2,314481,Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0,0,0,0,/wiki/Jurassic_World,http://www.imdb.com/title/tt0369610/,http://www.rottentomatoes.com/m/jurassic_world/,NaN,652198011,47.432927,73.000000,38.743326,7,7.50
4,tt0470752,Ex Machina,2015-04-24 00:00:00,UK,English,R,108,2015,7.7,194256,Ex Machina leans heavier on ideas than effects...,199,92,8.0,216,17,85,4.0,61929,0,0,0,0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,/wiki/Ex_Machina_(film),http://www.imdb.com/title/tt0470752/,http://www.rottentomatoes.com/m/ex_machina/,NaN,25440971,31.544133,31.000000,45.308693,2,7.85
7,tt0478970,Ant-Man,2015-07-17 00:00:00,USA,English,PG-13,117,2015,7.6,138824,"Led by a charming performance from Paul Rudd, ...",197,79,6.8,248,51,88,4.1,148048,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0,0,0,0,/wiki/Ant-Man_(film),http://www.imdb.com/title/tt0478970/,NaN,NaN,178636091,51.516363,67.333333,51.036277,26,7.90
8,tt0478970,Ant-Man,2015-07-17 00:00:00,USA,English,PG-13,117,2015,7.6,138824,"Led by a charming performance from Paul Rudd, ...",197,79,6.8,248,51,88,4.1,148048,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0,0,0,2,/wiki/Ant-Man_(film),http://www.imdb.com/title/tt0478970/,NaN,NaN,178636091,51.516363,67.333333,51.036277,26,7.90
10,tt0810819,The Danish Girl,2015-12-25 00:00:00,"UK, Germany, USA","English, German",R,120,2015,5.3,2539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.1,304,0,0,0,0,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,1,0,0,/wiki/The_Danish_Girl_(film),http://www.imdb.com/title/tt0810819/,http://www.rottentomatoes.com/m/The_Danish_Gir...,NaN,NaN,30.397230,25.500000,43.230664,18,6.75
